In [3]:
!pip install nltk
!pip install tflearn
!pip install tensorflow

  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=a759be921817359803ae114bf016942d1bb51395e7aa1477834f1cfe0a87fe8f
  Stored in directory: c:\users\sadhan\appdata\local\pip\cache\wheels\4a\d5\f8\9585b4a100c0fd73da204ee785457d67c85e1b9050f009a849
Successfully built tflearn


In [14]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sadhan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [15]:
import numpy
import tflearn
import tensorflow
import random
import json

In [16]:
with open("intents.json") as file:
    data = json.load(file)


In [17]:
words = []
labels = []
docs_x = []
docs_y = []

In [19]:
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent["tag"] not in labels:
        labels.append(intent["tag"])
        
        
words = [stemmer.stem(w.lower()) for w in words if w !="?"]
words = sorted(list(set(words)))

labels=sorted(labels)


In [22]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag=[]
    wrds = [stemmer.stem(w.lower()) for w in doc]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)


In [25]:
training = numpy.array(training)
output = numpy.array(output)

tensorflow.compat.v1.reset_default_graph() 

net = tflearn.input_data(shape=[None,len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]),activation = "softmax")
net = tflearn.regression(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [28]:
model = tflearn.DNN(net)

model.fit(training,output,n_epoch=1000, batch_size = 8,show_metric=True)
model.save("model.tflearn")


Training Step: 6999  | time: 0.011s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 0.0000 -- iter: 48/52
Training Step: 7000  | time: 0.013s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 0.0000 -- iter: 52/52
--
INFO:tensorflow:C:\Users\Sadhan\Desktop\Chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
